In [8]:
from config import *
df = get_prepared_df()

Index(['loop_type', 'home_structure', 'db_notation', 'whole_sequence',
       'helix_1_bps', 'helix_2_bps', 'helix_3_bps', 'helix_4_bps',
       'helix_5_bps', 'helix_6_bps',
       ...
       'helix_bps_bps', 'second_strand', 'first_strand', 'strand_nts_nts',
       'start', 'end', 'euler_x', 'euler_y', 'euler_z', 'planar'],
      dtype='object', length=182)


ValueError: Columns must be same length as key

In [2]:
!pip install pdbecif

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.9 MB/s eta 0:00:00


In [6]:
id_3ok4 = MMCIFParser(QUIET=True).get_structure(structure_id=0, 
                                                filename='raw_data/3ok4.cif')

In [13]:
obj = MMCIF2Dict().parse('raw_data/3ok4.cif')
bonds = pd.DataFrame(obj['3OK4']['_struct_conn'])

In [33]:
cols = ['conn_type_id', 
        'ptnr1_auth_comp_id', 'ptnr2_auth_comp_id',
        'ptnr1_auth_asym_id', 'ptnr1_auth_seq_id', 
        'ptnr2_auth_asym_id', 'ptnr2_auth_seq_id']
bond_info = bonds[cols]
bond_info['type'] = bond_info['conn_type_id']
bond_info['ptnr1'] = bond_info['ptnr1_auth_comp_id'].str[-1]
bond_info['ptnr2'] = bond_info['ptnr2_auth_comp_id'].str[-1]
bond_info['pos1'] = (bond_info['ptnr1_auth_asym_id'] + '-'
                     + bond_info['ptnr1_auth_seq_id'])
bond_info['pos2'] = (bond_info['ptnr2_auth_asym_id'] + '-'
                     + bond_info['ptnr2_auth_seq_id'])
bond_info = bond_info.drop(cols, axis=1)

In [35]:
bond_info.tail(200)

,type,ptnr1,ptnr2,pos1,pos2
340,hydrog,G,C,K-3,L-8
341,hydrog,U,A,K-4,L-7
342,hydrog,U,A,K-4,L-7
343,hydrog,A,U,K-5,L-6
344,hydrog,A,U,K-5,L-6
345,hydrog,A,U,K-6,L-5
346,hydrog,A,U,K-6,L-5
347,hydrog,U,A,K-7,L-4
348,hydrog,U,A,K-7,L-4
349,hydrog,G,C,K-8,L-3


In [20]:
bond_info.conn_type_id

0      covale
1      covale
2      covale
3      covale
4      covale
        ...  
535    hydrog
536    hydrog
537    hydrog
538    hydrog
539    hydrog
Name: conn_type_id, Length: 540, dtype: object